# SCC0270 - Redes Neurais e Aprendizado Profundo
### Aula 7 - Prática (Redes neurais convolutivas)
**Daniel Penna Chaves Bertazzo - 10349561**

# Exercício 1
A função de ativação *softmax* é dada pela seguinte fórmula:
<br>
$$ S(y_i) = \frac{e^{y_i}} {\sum_{j=1}^{K} e^{y_j}} $$
<br>
onde $K$ é o número de classes presentes no *dataset*. Tal função de ativação possui como entrada um vetor de $K$ elementos ($\in \mathbb{R}$) e o normaliza para uma distribuição de probabilidade (outro vetor com $K$ elementos, onde cada elemento representa uma probabilidade proporcional aos exponenciais dos valores de entrada). No contexto de redes neurais, é normalmente utilizada na camada de saída, pois pega como entrada os valores gerados pelas ativações dos neurônios da penúltima camada e os transforma em um vetor onde cada $S(y_i)$ representa a probabilidade da classe $i$ ser a predição correta para aquela instância do *dataset*. Por exemplo:
<br>
Suponha que possuímos um *dataset* com 3 classes diferentes e uma rede neural corretamente treinada para prever corretamente a classe de uma dada instância. Ao fornecer à rede tal exemplo, suponha que ela tenha como resultado o vetor:
<br><br>
$$ \begin{bmatrix} {0.02 \\
                    0.98 \\
                    0.00} \end{bmatrix} $$
<br>
onde cada linha representa uma classe $y$. Nesse caso, temos que
* $P(y=1) = 2\%$
* $P(y=2) = 98\%$
* $P(y=3) = 0\%$
<br>
Logo, de acordo com o modelo, a instância em questão tem 98% de probabilidade de ser da classe 2, sendo classificada como tal.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
# Carrega o dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
X_train.shape